In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [6]:
def calculate_RUL(X, upper_threshold=None):
    lt = X.groupby(['Unit'])['Time'].transform(max)
    rul = lt - X['Time']
    if upper_threshold:
        rul = np.where(rul > upper_threshold, upper_threshold, rul)
    return rul

In [7]:
train = pd.read_csv('train_FD001.csv')
test = pd.read_csv('test_FD001.csv')
rul = pd.read_csv('RUL_FD001.csv')


In [8]:
train_data = train

In [9]:
train

,Unit,Time,OperationalSetting1,OperationalSetting2,OperationalSetting3,Sensor-1Measurement,Sensor-2Measurement,Sensor-3Measurement,Sensor-4Measurement,Sensor-5Measurement,...,Sensor-12Measurement,Sensor-13Measurement,Sensor-14Measurement,Sensor-15Measurement,Sensor-16Measurement,Sensor-17Measurement,Sensor-18Measurement,Sensor-19Measurement,Sensor-20Measurement,Sensor-21Measurement
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100,38.30,23.1594
20628,100,198,0.0004,0.0000,100,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100,38.44,22.9333
20629,100,199,-0.0011,0.0003,100,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100,38.29,23.0640


In [10]:
train['rul'] = calculate_RUL(train, upper_threshold=135)
print(f'train.shape = {train.shape}')
train.head(2)

train.shape = (20631, 27)


,Unit,Time,OperationalSetting1,OperationalSetting2,OperationalSetting3,Sensor-1Measurement,Sensor-2Measurement,Sensor-3Measurement,Sensor-4Measurement,Sensor-5Measurement,...,Sensor-13Measurement,Sensor-14Measurement,Sensor-15Measurement,Sensor-16Measurement,Sensor-17Measurement,Sensor-18Measurement,Sensor-19Measurement,Sensor-20Measurement,Sensor-21Measurement,rul
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,135
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,135


remove sensors with constant values that we saw during Exploratory Data Analysis. For that, we create a transformer which drops features with variance lower than a threshold - 0

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold


class LowVarianceFeaturesRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0):
        self.threshold = threshold
        self.selector = VarianceThreshold(threshold=threshold)

    def fit(self, X):
        self.selector.fit(X)
        return self

    def transform(self, X):
        X_t = self.selector.transform(X)
        droped_features = X.columns[~self.selector.get_support()]
        print(f'Droped low variance features: {droped_features.to_list()}')
        return pd.DataFrame(X_t, columns=self.selector.get_feature_names_out())

Scaling per engine

In [35]:
sensor_names = ['Sensor-{}Measurement'.format(i) for i in range(1,22)] 

In [13]:
class ScalePerEngine(BaseEstimator, TransformerMixin):
    def __init__(self, n_first_cycles=20, sensors_columns=sensor_names):
        self.n_first_cycles = n_first_cycles
        self.sensors_columns = sensors_columns
    
    def fit(self, X):
        return self

    def transform(self, X):
        init_sensors_avg = X[X['Time'] <= self.n_first_cycles] \
            .groupby(by=['Unit'])[self.sensors_columns] \
            .mean() \
            .reset_index()

        X_t = X[X['Time'] > self.n_first_cycles].merge(
            init_sensors_avg,
            on=['Unit'], how='left', suffixes=('', '_init_v')
        )

        for SENSOR in self.sensors_columns:
            X_t[SENSOR] = X_t[SENSOR] - X_t['{}_init_v'.format(SENSOR)]

        drop_columns = X_t.columns.str.endswith('init_v')
        return X_t[X_t.columns[~drop_columns]]

tsfresh

In [14]:
from tsfresh.utilities.dataframe_functions import roll_time_series
from datetime import datetime

class RollTimeSeries(BaseEstimator, TransformerMixin):
    def __init__(self, min_timeshift, max_timeshift, rolling_direction):
        self.min_timeshift = min_timeshift
        self.max_timeshift = max_timeshift
        self.rolling_direction = rolling_direction
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        _start = datetime.now()
        print('Start Rolling TS')
        X_t = roll_time_series(
                    X, column_id='Unit', column_sort='Time', 
                    rolling_direction=self.rolling_direction,
                    min_timeshift=self.min_timeshift,
                    max_timeshift=self.max_timeshift)
        print(f'Done Rolling TS in {datetime.now() - _start}')
        return X_t

In [15]:
tsfresh_calc = {
    'mean_change': None,
    'mean': None,
    'standard_deviation': None,
    'root_mean_square': None,
    'last_location_of_maximum': None,
    'first_location_of_maximum': None,
    'last_location_of_minimum': None,
    'first_location_of_minimum': None,
    'maximum': None,
    'minimum': None,
    'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
    'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
    'cid_ce': [{'normalize': True}, {'normalize': False}],
    'autocorrelation': [{'lag': 0},
        {'lag': 1},
        {'lag': 2},
        {'lag': 3},],
    'partial_autocorrelation': [{'lag': 0},
        {'lag': 1},
        {'lag': 2},
        {'lag': 3},],
    'linear_trend': [
        {'attr': 'intercept'},
        {'attr': 'slope'},
        {'attr': 'stderr'}],
    'augmented_dickey_fuller': [{'attr': 'teststat'},
        {'attr': 'pvalue'},
        {'attr': 'usedlag'}],
    'linear_trend_timewise': [
        {'attr': 'intercept'},
        {'attr': 'slope'}],
    'lempel_ziv_complexity': [{'bins': 2},
        {'bins': 3},
        {'bins': 5},
        {'bins': 10},
        {'bins': 100}],
    'permutation_entropy': [{'tau': 1, 'dimension': 3},
        {'tau': 1, 'dimension': 4},
        {'tau': 1, 'dimension': 5},
        {'tau': 1, 'dimension': 6},
        {'tau': 1, 'dimension': 7}],
    'fft_coefficient': [
        {'coeff': 0, 'attr': 'abs'},
        {'coeff': 1, 'attr': 'abs'},
        {'coeff': 2, 'attr': 'abs'},
        {'coeff': 3, 'attr': 'abs'},
        {'coeff': 4, 'attr': 'abs'},
        {'coeff': 5, 'attr': 'abs'},
        {'coeff': 6, 'attr': 'abs'},
        {'coeff': 7, 'attr': 'abs'},
        {'coeff': 8, 'attr': 'abs'},
        {'coeff': 9, 'attr': 'abs'},
        {'coeff': 10, 'attr': 'abs'},],
    'fft_aggregated': [{'aggtype': 'centroid'},
        {'aggtype': 'variance'},
        {'aggtype': 'skew'},
        {'aggtype': 'kurtosis'}], 
}

In [16]:
from tsfresh import extract_features, select_features
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [17]:
class TSFreshFeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, calc=tsfresh_calc):
        self.calc = calc
        
    def _clean_features(self, X):
        old_shape = X.shape
        X_t = X.T.drop_duplicates().T
        print(f'Droped {old_shape[1] - X_t.shape[1]} duplicate features')

        old_shape = X_t.shape
        X_t = X_t.dropna(axis=1)
        print(f'Droped {old_shape[1] - X_t.shape[1]} features with NA values')
        return X_t
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        _start = datetime.now()
        print('Start Extracting Features')
        X_t = extract_features(
            X[['id', 'Time'] + X.columns[X.columns.str.startswith('Sensor-')].tolist()], 
            column_id='id', 
            column_sort='Time', 
            #column_value='TS values',
            default_fc_parameters=self.calc
            )
        print(f'Done Extracting Features in {datetime.now() - _start}')
        X_t = self._clean_features(X_t)
        return X_t

    
class CustomPCA(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=None, random_state=None):
        self.n_components = n_components
        self.random_state = random_state
        
    def fit(self, X):
        assert 'Unit' not in X.columns, "columns should be only features"
        self.ftr_columns = X.columns

        self.scaler = StandardScaler()
        X_sc = self.scaler.fit_transform(X[self.ftr_columns].values)

        self.pca = PCA(n_components=self.n_components, random_state=self.random_state)
        self.pca.fit_transform(X_sc)
        return self

    def transform(self, X):
        X_sc = self.scaler.transform(X[self.ftr_columns].values)
        X_pca = self.pca.transform(X_sc)        
        return pd.DataFrame(X_pca, index=X.index)

    

class TSFreshFeaturesSelector(BaseEstimator, TransformerMixin):
    def __init__(self, fdr_level=0.001):
        self.fdr_level = fdr_level

    def fit(self, X):
        rul = calculate_RUL(
            X.index.to_frame(name=['Unit', 'Time']).reset_index(drop=True),
            upper_threshold=135)

        X_t = select_features(
            X, rul, fdr_level=self.fdr_level
        )
        self.selected_ftr = X_t.columns
        
        print(f'Selected {len(self.selected_ftr)} out of {X.shape[1]} features: '
              f'{self.selected_ftr.to_list()}')
        return self

    def transform(self, X):
        return X[self.selected_ftr]

In [18]:
from sklearn.pipeline import Pipeline

In [ ]:
tsfresh_pipe = Pipeline([
    #Scaling
    ('scale-per-engine', ScalePerEngine(n_first_cycles=15, sensors_columns=sensor_names)),
    # Cleaning
    ('drop-low-variance', LowVarianceFeaturesRemover(threshold=0)),
    # Preprocessing
    ('roll-time-series', RollTimeSeries(min_timeshift=29, max_timeshift=29, rolling_direction=1)),
])

In [20]:
tsfresh_pipe2 = Pipeline([
    ('extract-tsfresh-features', TSFreshFeaturesExtractor(calc=tsfresh_calc)),
    ('PCA', CustomPCA(n_components=40)),
    ('features-selection', TSFreshFeaturesSelector(fdr_level=0.001)),
])

In [21]:
train_tsfresh_ftr = tsfresh_pipe.fit_transform(train_data)
train_tsfresh_ftr.head(2)

Droped low variance features: ['OperationalSetting3', 'Sensor-1Measurement', 'Sensor-5Measurement', 'Sensor-10Measurement', 'Sensor-16Measurement', 'Sensor-18Measurement', 'Sensor-19Measurement']
Start Rolling TS


Rolling: 100%|██████████| 20/20 [01:16<00:00,  3.84s/it]


Done Rolling TS in 0:01:17.978295


,Unit,Time,OperationalSetting1,OperationalSetting2,Sensor-2Measurement,Sensor-3Measurement,Sensor-4Measurement,Sensor-6Measurement,Sensor-7Measurement,Sensor-8Measurement,...,Sensor-11Measurement,Sensor-12Measurement,Sensor-13Measurement,Sensor-14Measurement,Sensor-15Measurement,Sensor-17Measurement,Sensor-20Measurement,Sensor-21Measurement,rul,id
0,1.0,16.0,0.0006,0.0005,-0.15,1.230667,3.746,0.0,-0.165333,-0.013333,...,-0.038,-0.555333,0.028,3.63,-0.015947,0.133333,-0.026,0.071733,135.0,"(1.0, 45.0)"
1,1.0,17.0,0.0002,0.0002,0.30,-1.789333,-0.804,0.0,-0.305333,-0.003333,...,-0.158,-0.155333,-0.002,4.79,0.044653,0.133333,-0.186,-0.051367,135.0,"(1.0, 45.0)"


In [22]:
train_tsfresh_ftr2 = tsfresh_pipe2.fit_transform(train_tsfresh_ftr)
train_tsfresh_ftr2.head(2)

Start Extracting Features


Feature Extraction: 100%|██████████| 20/20 [11:55<00:00, 35.78s/it]


Done Extracting Features in 0:12:34.329921
Droped 19 duplicate features
Droped 13 features with NA values
Selected 10 out of 40 features: [0, 2, 3, 4, 5, 1, 10, 28, 15, 30]


0         2         3         4         5         1   \
1.0 45.0 -11.017665 -0.151827  6.838070 -2.490502  0.089446  0.068197   
    46.0 -10.793015 -0.330131  6.704208 -2.569403 -0.120006  0.432618   

                10        28        15        30  
1.0 45.0  1.245446  2.993673  0.437758 -1.600702  
    46.0  1.356852  3.451939  0.290273 -1.234755

Rocket

In [29]:
from sklearn.preprocessing import StandardScaler


class CustomStandardScaler(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.scale_columns = X.columns[X.columns.str.startswith('Sensor-')| X.columns.str.startswith('OperationalSetting')]
        self.scaler = StandardScaler()
        self.scaler.fit(X[self.scale_columns])
        return self
    
    def transform(self, X):
        X_t = self.scaler.transform(X[self.scale_columns])
        df = pd.concat([
                X[[c for c in X.columns if c not in self.scale_columns]],
                pd.DataFrame(X_t, columns=self.scale_columns)
            ], axis=1).sort_values(['Unit', 'Time']).reset_index(drop=True)
        return df

In [2]:
#pip install sktime

In [31]:
from datetime import datetime

from sktime.datatypes._panel._convert import from_multi_index_to_nested


class TransformTS2Nested(BaseEstimator, TransformerMixin):
    '''
    See https://www.sktime.org/en/stable/examples/loading_data.html#Using-multi-indexed-pandas-DataFrames
    '''
    def fit(self, X):
        return self
    
    def transform(self, X):
        assert 'id' in X.columns, "X should have `id` column"
        X_mi = X.copy()
        X_mi.index = pd.MultiIndex.from_frame(X[['id', 'Time']])
        X_mi = X_mi.drop(columns=['rul', 'id'], errors='ignore')
        
        _start = datetime.now()
        print('Start Converting multi-index DF to sktime nested DF')
        X_nested = from_multi_index_to_nested(X_mi, instance_index='id')
        
        X_nested.index = pd.MultiIndex.from_frame(
            pd.DataFrame({'Unit': X_nested['Unit'].apply(lambda x: x.unique()[0]),
                          'Time': X_nested['Time'].apply(lambda x: x.iloc[-1])})
        )
        X_nested = X_nested.drop(columns=['Unit', 'Time'])
        print(f'Converted multi-index DF to sktime nested DF in {datetime.now() - _start}')

        return X_nested

In [33]:
from sktime.transformations.panel.rocket import Rocket

class RocketTransform(BaseEstimator, TransformerMixin):
    def __init__(self, num_kernels=10000, normalise=False, n_jobs=-1, random_state=None):
        self.num_kernels = num_kernels
        self.normalise = normalise
        self.n_jobs = n_jobs
        self.random_state = random_state

    def fit(self, X):
        self.rocket = Rocket(
            num_kernels=self.num_kernels, 
            normalise=self.normalise, 
            n_jobs=self.n_jobs,
            random_state=self.random_state)
        self.rocket.fit(X)
        return self
    
    def transform(self, X):
        rocket_t = self.rocket.transform(X)
        rocket_t.index = X.index
        return rocket_t

In [38]:
from sklearn.pipeline import Pipeline

rocket_pipe = Pipeline([
    
    # Scaling sensors values
    ('scale-per-engine', ScalePerEngine(n_first_cycles=15, sensors_columns=sensor_names)),
    ('scale', CustomStandardScaler()),
    # Cleaning constant features
    ('drop-low-variance', LowVarianceFeaturesRemover()),
    
    # Preprocessing for ROCKET
    ('roll-time-series', RollTimeSeries(min_timeshift=29, max_timeshift=29, rolling_direction=1)),
    ('nest-time-series', TransformTS2Nested()),
    
    # Features Engineering w ROCKET
    ('rocket', RocketTransform(num_kernels=10000, normalise=False, random_state=2021))
])

In [39]:
train_rocket_ftr = rocket_pipe.fit_transform(train)

train_rocket_ftr.shape

Droped low variance features: ['OperationalSetting3', 'Sensor-1Measurement', 'Sensor-5Measurement', 'Sensor-10Measurement', 'Sensor-16Measurement', 'Sensor-18Measurement', 'Sensor-19Measurement']
Start Rolling TS


Rolling: 100%|██████████| 20/20 [00:12<00:00,  1.62it/s]


Done Rolling TS in 0:00:13.540146
Start Converting multi-index DF to sktime nested DF
Converted multi-index DF to sktime nested DF in 0:09:53.726980


(16231, 20000)